Tercera parte: Spark <br>
Todos los ejercicios se relizan utilizando el API de RDDs de Spark.

In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext

import pandas as pd

import math

In [2]:
# Crear una instancia de SparkSession y SparkContext
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

23/10/06 22:21:53 WARN Utils: Your hostname, fe resolves to a loopback address: 127.0.1.1; using 192.168.68.71 instead (on interface enp0s3)
23/10/06 22:21:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/06 22:21:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/home/lu/.local/lib/python3.10/site-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
# Leer los archivos CSV.
df_apps = sqlContext.read.option("delimiter", ",").option("escape", '"').csv("./dataset/googleplaystore.csv", header=True, inferSchema=True)

df_reviews = sqlContext.read.option("delimiter", ",").option("escape", '"').csv("./dataset/googleplaystore_user_reviews.csv", header=True, inferSchema=True)

In [4]:
# Mostras los tipos de datos de los RDDs
df_apps.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



In [5]:
# Mostras los tipos de datos de los RDDs
df_reviews.printSchema()

root
 |-- App: string (nullable = true)
 |-- Translated_Review: string (nullable = true)
 |-- Sentiment: string (nullable = true)
 |-- Sentiment_Polarity: string (nullable = true)
 |-- Sentiment_Subjectivity: string (nullable = true)



Cuando se convierte un DataFrame en un RDD en Spark, los tipos de datos se mantienen

In [6]:
# Crear los RDDs a partir de los DataFrames
rdd_apps = df_apps.rdd
rdd_reviews = df_reviews.rdd

In [7]:
# Mostar los esquemas de los RDDs
print("Esquema de rdd_apps:")
rdd_apps.take(5)

Esquema de rdd_apps:


[Row(App='Photo Editor & Candy Camera & Grid & ScrapBook', Category='ART_AND_DESIGN', Rating=4.1, Reviews='159', Size='19M', Installs='10,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design', Last Updated='January 7, 2018', Current Ver='1.0.0', Android Ver='4.0.3 and up'),
 Row(App='Coloring book moana', Category='ART_AND_DESIGN', Rating=3.9, Reviews='967', Size='14M', Installs='500,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design;Pretend Play', Last Updated='January 15, 2018', Current Ver='2.0.0', Android Ver='4.0.3 and up'),
 Row(App='U Launcher Lite – FREE Live Cool Themes, Hide Apps', Category='ART_AND_DESIGN', Rating=4.7, Reviews='87510', Size='8.7M', Installs='5,000,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design', Last Updated='August 1, 2018', Current Ver='1.2.4', Android Ver='4.0.3 and up'),
 Row(App='Sketch - Draw & Paint', Category='ART_AND_DESIGN', Rating=4.5, Reviews='215644', Size='25M

In [8]:
# Mostar los esquemas de los RDDs
print("Esquema de rdd_reviews:")
rdd_reviews.take(5)

Esquema de rdd_reviews:


[Row(App='10 Best Foods for You', Translated_Review='I like eat delicious food. That\'s I\'m cooking food myself, case "10 Best Foods" helps lot, also "Best Before (Shelf Life)"', Sentiment='Positive', Sentiment_Polarity='1.0', Sentiment_Subjectivity='0.5333333333333333'),
 Row(App='10 Best Foods for You', Translated_Review='This help eating healthy exercise regular basis', Sentiment='Positive', Sentiment_Polarity='0.25', Sentiment_Subjectivity='0.28846153846153844'),
 Row(App='10 Best Foods for You', Translated_Review='nan', Sentiment='nan', Sentiment_Polarity='nan', Sentiment_Subjectivity='nan'),
 Row(App='10 Best Foods for You', Translated_Review='Works great especially going grocery store', Sentiment='Positive', Sentiment_Polarity='0.4', Sentiment_Subjectivity='0.875'),
 Row(App='10 Best Foods for You', Translated_Review='Best idea us', Sentiment='Positive', Sentiment_Polarity='1.0', Sentiment_Subjectivity='0.3')]

## Preprocesamiento de datos

In [9]:
# Buscamos en la columna "Reviews" el dato mal cargado, dado que esta columna solo acepta números enteros positivos.
# Para eso, filtramos los registros que no sean numéricos y los mostramos.
rdd_apps.filter(lambda x: not x["Reviews"].isdecimal()).collect() # se usa collect(), a pesar de que no es recomendable, porque la cantidad de registros que se espera es muy pequeña.


[Row(App='Life Made WI-Fi Touchscreen Photo Frame', Category='1.9', Rating=19.0, Reviews='3.0M', Size='1,000+', Installs='Free', Type='0', Price='Everyone', Content Rating=None, Genres='February 11, 2018', Last Updated='1.0.19', Current Ver='4.0 and up', Android Ver=None)]

Se considera que toda la fila esta mal cargada y se elimina.

In [10]:
# Eliminamos el registro que tiene el valor mal cargado.
rdd_apps = rdd_apps.filter(lambda x: x["Reviews"] != "3.0M")

## 2) Teniendo en cuenta las reviews que reciben las aplicaciones, devolver una (al azar) de la aplicación que haya recibido la mayor cantidad de reviews positivas. (⭐)

In [50]:
# Filtrar solo reseñas positivas (asumiremos que Sentiment es "Positive" en el segundo conjunto de datos)
positive_reviews_rdd = rdd_reviews.filter(lambda x: x['Sentiment'] == 'Positive')

# Filtrar solo valores numéricos en la columna "Reviews"
rdd_apps_numeric_reviews = rdd_apps.filter(lambda x: x['Reviews'].isdecimal())

# Convertir la columna "Reviews" a un tipo numérico
rdd_apps_numeric_reviews = rdd_apps_numeric_reviews.map(lambda x: (x['App'], int(x['Reviews'])))

# Obtener la aplicación con la mayor cantidad de reseñas positivas y su número de reseñas
app_with_most_positive_reviews = rdd_apps_numeric_reviews.reduce(lambda a, b: a if a[1] >= b[1] else b)

# Filtrar todas las reseñas de la aplicación con la mayor cantidad de reseñas positivas
reviews_of_app_with_most_positive_reviews = positive_reviews_rdd.filter(lambda x: x['App'] == app_with_most_positive_reviews[0])

# Seleccionar una reseña al azar de esa aplicación sin usar collect()
random_review = reviews_of_app_with_most_positive_reviews.takeSample(False, 1)

# Imprimir la aplicación y la reseña seleccionada al azar
print("Aplicación con más reseñas positivas:", app_with_most_positive_reviews[0])
print("Reseña seleccionada al azar:")
print("Translated_Review:", random_review[0]['Translated_Review'])
print("Sentiment:", random_review[0]['Sentiment'])

Aplicación con más reseñas positivas: Facebook
Reseña seleccionada al azar:
Translated_Review: Would great change constantly. The videos inconsistent. One video take video tab another go full screen preferred.
Sentiment: Positive


## 31) ¿Cuál es la app con mayor cantidad de instalaciones? (⭐)

La columna "Installs" tiene valores como "1,000,000+" o "100,000,000+" que no se pueden convertir a números. Se limpian los datos y se convierten a números.

In [51]:
# Convertir la columna "Installs" a un tipo numérico
rdd_apps_installs = rdd_apps.map(lambda x: (x['App'], int(x['Installs'].replace('+', '').replace(',', ''))))

# Obtener la aplicación con la mayor cantidad de instalaciones y su número de instalaciones
app_with_most_installs = rdd_apps_installs.reduce(lambda a, b: a if a[1] >= b[1] else b)

# Imprimir la aplicación con la mayor cantidad de instalaciones
print("Aplicación con más instalaciones:", app_with_most_installs[0])


Aplicación con más instalaciones: Google Play Books


## 10) Cual es la categoría con mayor promedio de apps que hayan sido al menos 1 vez calificadas como positivas (⭐⭐)

Ejemplo con números hipotéticos:

Supongamos que en la categoría "Juegos" hay 100 aplicaciones únicas.

Luego encontramos que de esas 100 aplicaciones, 60 tienen al menos 1 reseña positiva.

Entonces los cálculos serían:

- Total de apps únicas en la categoría: 100
- Apps con al menos 1 reseña positiva: 60
- Porcentaje de apps positivas: 60/100 = 0.6 (60%)

Así que para la categoría "Juegos" el porcentaje de apps con al menos una reseña positiva es del 60%.

Este mismo cálculo se hace para cada categoría:

- Contar el total de apps únicas en esa categoría
- Contar cuántas tienen al menos 1 reseña positiva
- Dividir las positivas sobre el total para obtener el porcentaje

Finalmente, se ordenan las categorías por ese porcentaje de mayor a menor, y se obtiene la categoría con el mayor porcentaje, que sería la que en promedio tiene más probabilidades de que sus apps tengan reseñas positivas.

In [11]:
# Filtrar las revisiones con sentimiento positivo
rdd_positive_reviews = rdd_reviews.filter(lambda x: x["Sentiment"] == "Positive")

# Mapear el RDD de revisiones para obtener (App, 1) como clave-valor
rdd_positive_counts = rdd_positive_reviews.map(lambda x: (x["App"], 1))

# Reducir por clave (App) para contar las aplicaciones únicas con al menos una reseña positiva
rdd_app_positive_counts = rdd_positive_counts.reduceByKey(lambda a, b: a)

In [12]:
# Realizar un join entre los RDDs de aplicaciones y conteo de revisiones positivas
rdd_app_category_join = rdd_apps.map(lambda x: (x["App"], x["Category"])).distinct()

In [13]:
# Calcular el total de aplicaciones en cada categoría
rdd_category_counts = rdd_app_category_join.map(lambda x: (x[1], 1)).reduceByKey(lambda a, b: a + b)
rdd_result = rdd_app_category_join.join(rdd_app_positive_counts)

In [14]:
# Calcular el total de aplicaciones con al menos una reseña positiva en cada categoría
rdd_category_positive_counts = rdd_result.map(lambda x: (x[1][0], 1)).reduceByKey(lambda a, b: a + b)

In [15]:
# Calcular los porcentajes de aplicaciones con al menos una reseña positiva en cada categoría
rdd_category_positive_percentages = rdd_category_positive_counts.join(rdd_category_counts).map(lambda x: (x[0], x[1][0] / x[1][1]))

In [16]:
# Encontrar la categoría con el mayor porcentaje de aplicaciones positivas
rdd_category_positive_percentages.reduce(lambda a, b: a if a[1] >= b[1] else b)

('ENTERTAINMENT', 0.24509803921568626)

## 18) Calcule el tamaño promedio de las aplicaciones por versión de Android, sin tener en cuenta las aplicaciones que varían en tamaño según dispositivo. (⭐⭐)

In [ ]:
# # Calcule el tamaño promedio de las aplicaciones por versión de Android, sin tener en cuenta las aplicaciones que varían en tamaño según dispositivo
# Al se una cadena la columna Size, que tiene valores como "Varies with device", "1.1M" y "175k"

# Filtrar las aplicaciones que varían en tamaño según dispositivo
rdd_apps_fixed_size = rdd_apps.filter(lambda x: x["Size"] != "Varies with device")

## 19) Para cada categoría, indicar cuál es la aplicación que tiene mayor cantidad de reviews con sentimiento negativo (⭐⭐)

## Borrar

In [76]:
from pyspark.sql import SparkSession

# Crear una instancia de SparkSession
spark = SparkSession.builder.appName("AppInstalls").getOrCreate()

# Leer el archivo CSV directamente como un RDD
rdd_apps = spark.sparkContext.textFile("./dataset/googleplaystore.csv")

# Filtrar y dividir las líneas correctamente
header = rdd_apps.first()
rdd_apps = rdd_apps.filter(lambda line: line != header).map(lambda line: line.split(','))

# Función para manejar la conversión a número con manejo de errores
def safe_int(value):
    try:
        return int(value)
    except ValueError:
        return 0

# Obtener la aplicación con la mayor cantidad de instalaciones
app_with_most_installs = rdd_apps.map(lambda x: (x[0], safe_int(x[5].replace('+', '').replace(',', '')))).max(key=lambda x: x[1])

# Imprimir la aplicación con la mayor cantidad de instalaciones
print("Aplicación con más instalaciones:", app_with_most_installs[0])

23/10/05 11:13:34 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


Aplicación con más instalaciones: "360 Security - Free Antivirus


In [83]:
# ejemplo de uso de safe_int
a = 'Varies'
b = a.replace('+', '').replace(',', '')

print(safe_int('1,000,000+'))
print(safe_int('1,000,000'))
print(safe_int('Varies with device'))

print(safe_int(b))
c = safe_int(b)

0
0
0
0
